# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
file = "../output_data/cities.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,sola,-13.88,167.55,81.50,80.0,60.0,19.60,VU,1.604180e+09
1,pangnirtung,66.15,-65.71,13.03,79.0,100.0,5.17,CA,1.604180e+09
2,butaritari,3.07,172.79,81.39,75.0,100.0,15.57,KI,1.604180e+09
3,mahebourg,-20.41,57.70,66.20,82.0,75.0,8.05,MU,1.604180e+09
4,atuona,-9.80,-139.03,78.53,75.0,12.0,17.18,PF,1.604180e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
gmaps.configure(api_key=g_key)

In [42]:
latLongDF = weather_df.loc[:, ["Latitude", "Longitude"]]
coordinatesDF = latLongDF.to_records(index=False)
coordinates = list(coordinatesDF)
coordinates

[(-13.88, 167.55),
 (66.15, -65.71),
 (3.07, 172.79),
 (-20.41, 57.7),
 (-9.8, -139.03),
 (-21.21, -159.78),
 (19.73, -155.09),
 (-6.03, -37.02),
 (-21.36, 55.77),
 (47.56, 46.72),
 (55.74, -97.86),
 (69.38, 178.42),
 (-46.13, 167.68),
 (-33.93, 18.42),
 (-46.19, 168.86),
 (-53.15, -70.92),
 (65.28, -126.83),
 (-42.88, 147.33),
 (-24.87, 113.63),
 (-8.47, 140.33),
 (14.51, -92.19),
 (70.63, 147.92),
 (71.29, -156.79),
 (22.29, 114.16),
 (-12.04, -77.03),
 (-11.78, 19.92),
 (17.02, 54.09),
 (32.69, 128.84),
 (48., 19.2),
 (4.87, -2.24),
 (64.38, -173.3),
 (-21.15, 149.2),
 (-34.42, 19.23),
 (-42.78, 147.06),
 (33.21, -87.57),
 (78.22, 15.64),
 (65.58, -171.),
 (42.6, -73.97),
 (-11.9, 33.6),
 (-23.12, -134.97),
 (64.25, -15.21),
 (62.46, -114.35),
 (40.74, -114.07),
 (71.69, 128.87),
 (63.76, 121.62),
 (23.45, -110.22),
 (-2.57, 150.8),
 (27.34, 117.48),
 (-34.58, 150.86),
 (66.94, -53.67),
 (-34.53, 20.04),
 (22.08, -159.32),
 (-33.65, 115.33),
 (-24.79, -50.01),
 (47.69, -114.16),
 (5

In [53]:
tempDF = weather_df.loc[:,["Temperature"]]
Temp = list(TempsDF)
Temp

rec.array([( 81.5 ,), ( 13.03,), ( 81.39,), ( 66.2 ,), ( 78.53,),
           ( 75.2 ,), ( 83.23,), ( 85.64,), ( 61.11,), ( 46.  ,),
           ( 19.4 ,), ( 17.56,), ( 57.  ,), ( 60.17,), ( 57.  ,),
           ( 57.2 ,), (  3.2 ,), ( 57.96,), ( 69.8 ,), ( 79.03,),
           ( 86.11,), ( -8.  ,), ( 21.2 ,), ( 69.33,), ( 64.71,),
           ( 64.2 ,), ( 73.4 ,), ( 68.14,), ( 46.08,), ( 78.49,),
           ( 22.32,), ( 76.33,), ( 57.  ,), ( 59.32,), ( 71.65,),
           ( 24.8 ,), ( 20.88,), ( 38.17,), ( 64.51,), ( 75.67,),
           ( 42.96,), ( 20.82,), ( 60.8 ,), (  2.41,), ( -4.49,),
           ( 82.  ,), ( 84.65,), ( 62.46,), ( 60.76,), ( 20.62,),
           ( 57.2 ,), ( 83.52,), ( 57.18,), ( 56.66,), ( 48.49,),
           ( 36.46,), ( 75.2 ,), ( 66.54,), ( 12.2 ,), ( 61.83,),
           ( -2.07,), ( 18.68,), ( 30.65,), ( 74.52,), (  7.57,),
           ( 28.81,), ( 32.74,), ( 66.2 ,), (-13.86,), ( 50.  ,),
           ( 75.24,), ( 69.8 ,), ( 56.57,), ( 78.67,), ( 44.49,),
          

In [47]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=Temp, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

TraitError: (81.5,) is not a valid weight. Weights must be floats

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
